In [ ]:
# chr(0)
# print(chr(0))
# # "this is a test" + chr(0) + "string"
# print("this is a test" + chr(0) + "string")

# # Output: this is a teststring

In [5]:
# UTF 8 Encode-Decode
test_string = "Hello! How are you? ఏం చేస్తున్నావ్?"
utf8_encode = test_string.encode("utf-8")
print(utf8_encode)
print(type(utf8_encode))
list(utf8_encode) # Get the byte values for the encoded string (integers from 0 to 255).
# One byte does not necessarily correspond to one Unicode character!
print(len(test_string))
print(len(utf8_encode))
print(utf8_encode.decode(("utf-8")))

b'Hello! How are you? \xe0\xb0\x8f\xe0\xb0\x82 \xe0\xb0\x9a\xe0\xb1\x87\xe0\xb0\xb8\xe0\xb1\x8d\xe0\xb0\xa4\xe0\xb1\x81\xe0\xb0\xa8\xe0\xb1\x8d\xe0\xb0\xa8\xe0\xb0\xbe\xe0\xb0\xb5\xe0\xb1\x8d?'
<class 'bytes'>
36
64
Hello! How are you? ఏం చేస్తున్నావ్?


Question: 

Deliverable: Write a function that, given path to an input text file, trains a (byte-level) BPE
tokenizer. 

Your BPE training function should handle (at least) the following input parameters:

*input_path*: str Path to a text file with BPE tokenizer training data.

*vocab_size*: int A non-negative integer that defines the maximum final vocabulary size (including the initial byte vocabulary, vocabulary items produced from merging, and any special tokens).

*special_tokens*: list[str] A list of strings to add to the vocabulary. These special tokens do not otherwise affect BPE training.

Your BPE training function should return the resulting vocabulary and merges:

*vocab*: dict[int, bytes] The tokenizer vocabulary, a mapping from int (token ID in the vocabu-
lary) to bytes (token bytes).
*merges*: list[tuple[bytes, bytes]] A list of BPE merges produced from training. Each list
item is a tuple of bytes (<token1>, <token2>), representing that <token1> was merged with
<token2>. The merges should be ordered by order of creation.

To test your BPE training function against our provided tests, you will first need to implement the test adapter at [adapters.run_train_bpe]. Then, run pytest tests/test_train_bpe.py. Your implementation should be able to pass all tests.

In [6]:
input_text = """this is a test
this is another test
byte pair encoding test"""

In [24]:
import re
from collections import Counter
from typing import List, Tuple, Dict


def get_stats(text: List[List[bytes]]) -> Counter:
    """Get the frequencies of byte pairs in the text."""
    pairs = Counter()
    for word in text:
        for i in range(len(word) - 1):
            pair = (word[i], word[i + 1])
            pairs[pair] += 1
    return pairs


def merge_vocab(pair: Tuple[bytes, bytes], vocab: Dict[Tuple[bytes], int]) -> Dict[Tuple[bytes], int]:
    """Merge the most frequent byte pair into the vocabulary."""
    bigram = ' '.join(map(str, pair))
    replacement = bytes([pair[0] + pair[1]])
    
    new_vocab = {}
    for word, freq in vocab.items():
        new_word = []
        i = 0
        while i < len(word) - 1:
            if word[i:i+2] == pair:
                new_word.append(replacement)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        if i < len(word):
            new_word.append(word[i])
        new_vocab[tuple(new_word)] = freq
    return new_vocab


def train_bpe(input_path: str, vocab_size: int, special_tokens: List[str]) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
    """Train a byte-level BPE tokenizer."""
    # Step 1: Read the input text file
    with open(input_path, 'rb') as f:
        text = [list(line.strip()) for line in f]

    # Step 2: Initialize the vocabulary with individual characters (bytes)
    vocab = Counter(tuple(line) for line in text)
    
    # Step 3: Add special tokens to the vocabulary
    special_tokens = [t.encode() for t in special_tokens]
    
    # Add special tokens with a count of 0 to the vocabulary
    for token in special_tokens:
        vocab[tuple([token])] = 0
    
    # Step 4: Perform BPE merging
    merges = []
    while len(vocab) < vocab_size:
        pairs = get_stats(list(vocab.keys()))
        
        if not pairs:
            break
        
        # Get the most frequent pair
        best_pair = pairs.most_common(1)[0][0]
        
        # Merge the pair
        vocab = merge_vocab(best_pair, vocab)
        merges.append(best_pair)
    
    # Step 5: Prepare the final vocab dictionary with int->byte mapping
    final_vocab = {i: token for i, token in enumerate(vocab, start=len(special_tokens))}
    
    return final_vocab, merges




# Example of running the function
if __name__ == "__main__":
    input_file = "input_text.txt"
    vocab_size = 10000
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]"]

    vocab, merges = train_bpe(input_file, vocab_size, special_tokens)
    
    print("Vocabulary:", vocab)
    print("Merges:", merges)


TypeError: can't concat int to bytes